## **Introduction**

This notebook focuses on enhancing large language models (LLMs) to become more reliable and capable assistants by integrating tools, improving information accuracy, and expanding their functionality. Key topics include:

1. **Tool Integration**: LLMs can be augmented with tools to access real-time data, perform specific tasks (e.g., web searches, database queries), and overcome limitations in domain-specific or up-to-date knowledge.
   
2. **Research Assistant Application**: An example application demonstrates how connecting external data sources improves LLM performance by grounding responses in factual, current information.

3. **Information Extraction**: Techniques for extracting structured information from unstructured documents are explored, enabling LLMs to better process and analyze complex data.

4. **Fact-Checking**: Methods to mitigate hallucinations and inaccuracies in LLM outputs are discussed, including automatic fact-checking against evidence to ensure correctness and reduce misinformation.

5. **Reasoning Strategies**: Advanced reasoning approaches are applied to further enhance the capabilities of LLMs in solving complex problems.

This section explores how tools can enhance the capabilities of large language models (LLMs) by addressing their limitations, particularly in areas like complex calculations or accessing external data. Below is a concise summary:

### Key Points on Tool Use:
1. **Tool Integration**:
   - LLMs struggle with tasks requiring precise computations or logical reasoning, where specialized tools (e.g., calculators) excel.
   - Combining LLMs with tools improves their problem-solving abilities and enables them to perform exact computations.

2. **Tool Components**:
   - **Name**: A unique identifier for the tool within an agent's toolkit.
   - **Function**: The executable logic or action the tool performs.
   - **Description**: A textual explanation that helps the LLM (agent) select the appropriate tool for a task.
   - **Args Schema**: An optional Pydantic BaseModel schema that provides parameter validation and examples for inputs.
   - **Return Direct Flag**: Indicates whether the tool's result should be returned directly to the user.

3. **Example: WikipediaQueryRun Tool**:
   - Demonstrates the use of a built-in LangChain tool, `WikipediaQueryRun`, which wraps around Wikipedia for querying general knowledge.
   - Initialization involves configuring parameters like `top_k_results` and `doc_content_chars_max`.
   - The tool's attributes (`name`, `description`, `args`, `return_direct`) provide clarity on its purpose and usage.
   - It can be invoked with either a dictionary input or a simple string query.

4. **Custom Tools**:
   - While LangChain offers many pre-built tools, creating custom tools allows for tailored functionality and deeper understanding of the underlying mechanics.


In [1]:
# Import necessary modules
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Initialize the tool
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# Inspect tool properties
print(tool.name)  # 'wikipedia'
print(tool.description)  # Describes the tool's utility and expected input
print(tool.args)  # JSON schema for inputs
print(tool.return_direct)  # False, indicating results are processed further

# Run the tool
result = tool.run("langchain")  # Query Wikipedia for "langchain"
print(result)  # Returns a summary of the LangChain page

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'title': 'Query', 'type': 'string'}}
False
Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


## Defining custom tools
To define custom tools in LangChain, you can follow approaches such as these:

@tool decorator
Subclassing BaseTool
StructuredTool dataclass

This section introduces the concept of using Python's **decorator syntax** to define tools in a more concise and intuitive way. Below is a summary of how this works:

### Key Points on Tool Decorators:
1. **Decorator Syntax**:
   - Tools can be defined using Python decorators, which simplify the process of creating tools.
   - The `@tool` decorator is used to mark a function as a tool.

2. **Default Behavior**:
   - By default, the decorator uses the function's name as the tool's name.
   - The function's docstring serves as the tool's description, making it mandatory to include a clear and descriptive docstring.

3. **Advantages**:
   - Simplifies the creation of tools by embedding the tool's logic directly within a function.
   - Encourages clarity through the use of docstrings, ensuring that the tool's purpose is well-documented.

### Example of Using the `@tool` Decorator:
Here’s an example of how to define a custom tool using the decorator:




In [5]:
from langchain.tools import tool
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

search("What's the best application framework for LLMs?")

'LangChain'

In [ ]:
@tool
def search_wikipedia(query: str) -> str:
    """
    Searches Wikipedia for the given query and returns a summary of the top result.
    Useful for answering general knowledge questions about people, places, companies, historical events, etc.
    """
    # Simulated Wikipedia search logic (replace with actual implementation)
    return f"Summary of Wikipedia page for '{query}'"

# Using the tool
result = search_wikipedia("LangChain")
print(result)  # Output: Summary of Wikipedia page for 'LangChain'

Summary of Wikipedia page for 'LangChain'


We can customize the tool name and JSON args by passing them into the tool decorator:

In [7]:
from langchain.pydantic_v1 import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")
@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

search("What's the best application framework for LLMs?")

'LangChain'

### Subclassing BaseTool 

This provides the most flexibility for defining tools, allowing customization of behavior, complex logic, asynchronous operations, and error handling. This approach involves creating a class that includes tool metadata (name, description), input schema definitions, and synchronous/asynchronous execution methods. Below is a concise summary:

Key Points on Subclassing BaseTool:
Customization : Offers full control over tool behavior, beyond simple function execution.
Complex Logic : Enables implementation of advanced or multi-step processes.
Asynchronous Support : Handles async operations, useful for tasks like API calls.
Error Handling : Allows tailored error management and logging specific to the tool.
Structure : Includes metadata (name, description), input schemas, and execution methods.

In [9]:
from typing import Optional, Type 
from langchain.tools import BaseTool 
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun, CallbackManagerForToolRun,
)
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")
class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"
    args_schema: Type[BaseModel] = SearchInput
    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Use the tool."""
        return "LangChain"
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
search = CustomSearchTool()
search("What's the most popular tool for writing LLM apps?")

'LangChain'

In this example:

We define a custom input schema (SearchInput)
The tool’s name and description are class attributes
The _run method implements the synchronous tool execution
The _arun method is a placeholder for asynchronous execution
Both methods include optional callback managers for advanced control
This approach requires more code but provides flexibility for complex tool implementations and integration with LangChain’s broader ecosystem.

### StructuredTool `dataclass`

StructuredTool offers a convenient way to define tools for Langchain workflows. It provides a balance between inheriting from the base BaseTool class (more complex) and simply using a decorator (less functionality).

In [12]:
from langchain.tools import StructuredTool
def search_function(query: str):
    return "LangChain"
search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer questions about current events",
)
search("Which framework has hundreds of integrations to use with LLMs?")

'LangChain'

This preceding code defines a simple function search_function that always returns "LangChain". Then, it creates a StructuredTool object named search by specifying the function, name, and description. Finally, it calls the run method on the search tool with a query string, demonstrating how to use the tool.

StructuredTool allows you to define a custom input schema using a BaseModel subclass. This provides better type checking and documentation for your tool’s inputs:

In [22]:
from pydantic.v1 import BaseModel, Field  # <-- Use Pydantic v1 API
from langchain.tools import StructuredTool

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="Multiply two numbers",
    args_schema=CalculatorInput,  # Make sure this is correctly passed
    return_direct=True,
)

result = calculator.run({"a": 1_000_000_000, "b": 2})
print(result)


2000000000


### Defining and Using a Structured Tool in LangChain  

In the example above, we define a `CalculatorInput` class to specify the expected input format—two integers. This class ensures proper type validation using Pydantic. The `multiply` function then utilizes this schema to enforce type safety.  

We also set `return_direct=True`, which allows the function’s output to be returned immediately without additional processing. Finally, we call the `run` method with a dictionary containing values for `a` and `b`, executing the multiplication operation.  

While `StructuredTool` simplifies tool creation, handling potential errors during execution is essential. The next section will explore strategies for implementing error-handling mechanisms in LangChain tools.  

### Why We Used `pydantic.v1`  

LangChain was originally built with **Pydantic v1**, but recent versions of Pydantic introduced breaking changes in v2. Some parts of LangChain **still expect Pydantic v1-style schemas**, which led to validation errors when using `pydantic` v2 directly.  

To maintain compatibility, we explicitly imported from `pydantic.v1`:  

```python
from pydantic.v1 import BaseModel, Field
```

This ensures that LangChain functions correctly interpret the input schema and avoid validation errors when using `StructuredTool`.  

As LangChain continues evolving, future versions may fully support Pydantic v2, eliminating the need for this workaround. Until then, using `pydantic.v1` remains the most reliable solution for structured tool definitions.

### Error handling
Additionally, LangChain provides a way to handle tool errors. When a tool encounters an error and the exception is not caught, the agent will stop executing. To allow the agent to continue execution, you can raise a ToolException and set handle_tool_error accordingly:

In [23]:
from langchain_core.tools import ToolException
def search_tool(s: str):
    raise ToolException("The search tool is not available.")
search = StructuredTool.from_function(
    func=search_tool,
    name="Search_tool",
    description="A bad tool",
    handle_tool_error=True,
)
search("Search the internet and compress everything into a paragraph!")

'The search tool is not available.'

Executing this should give you an error: `'The search tool is not available.'`

You can also define a custom way to handle the tool error by setting `handle_tool_error` to a function that takes a `ToolException` as a parameter and returns a str value.

In [24]:
from langchain_core.tools import ToolException, StructuredTool

# Define a custom error handler
def custom_error_handler(error: ToolException) -> str:
    return f"Oops! Something went wrong: {error}"

# Define a faulty tool
def search_tool(s: str):
    raise ToolException("The search tool is not available.")

# Create the tool with custom error handling
search = StructuredTool.from_function(
    func=search_tool,
    name="Search_tool",
    description="A bad tool",
    handle_tool_error=custom_error_handler,  # Pass custom error handler
)

# Run the tool
result = search.run("Search the internet and compress everything into a paragraph!")
print(result)


Oops! Something went wrong: The search tool is not available.


### Building a Research Assistant with LangChain  

We will learn how to create a powerful research assistant by integrating an LLM with external tools for information gathering and query handling. We hope to break down LangChain agents, equipping ourself with the essentials to build an intelligent, efficient assistant.

Let’s start by defining a function to create a Langchain agent with a few basic tools:

In [25]:
from langchain.agents import (
    AgentExecutor, load_tools, create_react_agent
)
from langchain.chat_models import ChatOpenAI
def load_agent() -> AgentExecutor:
    llm = ChatOpenAI(temperature=0, streaming=True)
    tools = load_tools(
        tool_names=["ddg-search", "arxiv", "wikipedia"],
        llm=llm
    )
    return AgentExecutor(
        agent=create_react_agent(llm=llm, tools=tools), tools=tools
    )

The `load_agent()` function initializes a LangChain agent with a **ChatOpenAI** model, enabling **streaming responses** for a better user experience. It loads tools like **DuckDuckGo (privacy-focused search), arXiv (academic research), and Wikipedia (entity information)** to enhance the agent’s capabilities. The agent is built using the **ReAct (Reasoning + Acting) architecture** for decision-making.

The discussion also mentions **alternative tools** available in LangChain, such as **Wolfram Alpha (math and logic), Google/Bing search, Tavily Search API (optimized for LLMs), and Open-Meteo (weather data)**.

Finally, the text suggests **deploying the agent as a Streamlit app** to create an interactive web interface, leveraging Streamlit’s simplicity and ML workflow optimization.

---


**Overall Purpose**

The following code constructs an interactive chatbot application using the Streamlit library. The chatbot leverages the power of a large language model (LLM) combined with external tools like arXiv and Wikipedia to provide comprehensive and informative responses to user queries. The underlying framework for the chatbot's reasoning and action selection is the ReAct (Reasoning and Acting) framework.

**Code Breakdown**

1. **Initialization and Setup**

   ```python
   from config import set_environment  # Import a custom function (presumably for setting API keys and environment variables)

   set_environment()  # Call the function to configure the environment

   import streamlit as st  # Import the Streamlit library for building the web application

   from langchain_community.callbacks.streamlit import StreamlitCallbackHandler  # Import a callback handler for integrating with Streamlit

   from langchain.prompts import PromptTemplate  # Import PromptTemplate for structuring prompts
   from langchain.agents import AgentExecutor, load_tools, create_react_agent  # Import components for creating and managing agents
   from langchain_community.chat_models import ChatOpenAI  # Import the ChatOpenAI language model
   ```

   - These lines import the necessary libraries and modules.
   - `set_environment()` is assumed to handle any required environment configuration, such as setting API keys for the language model and tools.

2. **`load_agent()` Function**

   ```python
   def load_agent() -> AgentExecutor:
       """
       This function initializes and loads a ReAct agent with specified tools and prompt template.

       Returns:
           AgentExecutor: An agent executor ready to handle user requests.
       """
       llm = ChatOpenAI(temperature=0.2, streaming=True)  # Initialize the language model with a temperature of 0.2 and streaming enabled.
       tools = load_tools(tool_names=["arxiv", "wikipedia"], llm=llm)  # Load the arXiv and Wikipedia tools for the agent to use.

       # Define the prompt template for the agent, instructing it on the format for its responses.
       prompt = PromptTemplate(
           input_variables=["input", "tools", "tool_names", "agent_scratchpad"],
           template=(
               "You are an assistant with access to the following tools: {tool_names}\n\n"
               "{tools}\n\n"
               "Use the following format *EXACTLY*:\n"
               "Question: the input question\n"
               "Thought: consider what to do\n"
               "Action: the action to take, should be one of {tool_names}\n"
               "Action Input: the input to the action\n"
               "Observation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\n"
               "Thought: I now know the final answer\n"
               "Final Answer: the final answer to the original input question\n\n"
               "Question: {input}\n"
               "{agent_scratchpad}"
           ),
       )

       # Create the ReAct agent and return an agent executor.
       agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)
       return AgentExecutor(agent=agent, tools=tools)
   ```

   - This function encapsulates the logic for creating and configuring the ReAct agent.
   - `llm = ChatOpenAI(temperature=0.2, streaming=True)`: Initializes the `ChatOpenAI` language model.
     - `temperature=0.2`: Controls the randomness of the model's output. A lower temperature makes the output more deterministic.
     - `streaming=True`: Enables streaming of the model's output, allowing for a more interactive user experience.
   - `tools = load_tools(tool_names=["arxiv", "wikipedia"], llm=llm)`: Loads the arXiv and Wikipedia tools. These tools allow the agent to access and retrieve information from these sources.
   - `prompt = PromptTemplate(...)`: Defines a `PromptTemplate` that structures the instructions and format for the agent's responses. The template emphasizes a clear ReAct format, where the agent explicitly states its thought process, actions, and observations.
   - `agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)`: Creates the ReAct agent using the provided language model, tools, and prompt template.
   - `return AgentExecutor(agent=agent, tools=tools)`: Returns an `AgentExecutor` that manages the execution of the agent and its interactions with the tools.

3. **Streamlit Application Logic**

   ```python
   chain = load_agent()  # Initialize the agent executor

   # Get user input from the Streamlit chat interface
   if prompt := st.chat_input():
       st.chat_message("user").write(prompt)  # Display the user's message
       with st.chat_message("assistant"):  # Context for the assistant's response
           st_callback = StreamlitCallbackHandler(st.container())  # Set up a callback handler for Streamlit
           response = chain.invoke(
               {"input": prompt}, callbacks=[st_callback], handle_parsing_errors=True  # Invoke the agent with the user's prompt, callbacks, and error handling
           )
           print(f"Response: {response}")  # Print the complete response from the agent
           if "agent_outcome" in response and response["agent_outcome"] is not None:  # Check if the response contains agent outcome information
               print(f"Agent Outcome: {response['agent_outcome']}")  # Print the agent outcome details
           st.write(response["output"])  # Display the agent's output in the chat interface
   ```

   - `chain = load_agent()`: Initializes the agent executor by calling the `load_agent()` function.
   - `if prompt := st.chat_input():`: Checks if the user has provided input in the Streamlit chat interface.
   - `st.chat_message("user").write(prompt)`: If there is user input, display it in the chat as a user message.
   - `with st.chat_message("assistant"):`: Sets the context for the assistant's response in the chat.
   - `st_callback = StreamlitCallbackHandler(st.container())`: Creates a `StreamlitCallbackHandler` to integrate the agent's intermediate steps (thoughts, actions, observations) with the Streamlit application, allowing for a more interactive display of the agent's reasoning process.
   - `response = chain.invoke(...)`: Invokes the agent (using the `AgentExecutor`) with the user's prompt and additional parameters:
     - `{"input": prompt}`: Provides the user's input as a dictionary with the key "input."
     - `callbacks=[st_callback]`: Includes the Streamlit callback handler to display the agent's thinking process.
     - `handle_parsing_errors=True`: Enables error handling to allow the agent to recover from potential parsing errors during its response generation.
   - `print(f"Response: {response}")`: Prints the complete response dictionary for debugging and inspection.
   - `if "agent_outcome" in response and response["agent_outcome"] is not None:`: Checks if the response dictionary contains information about the agent's outcome and prints it if available.
   - `st.write(response["output"])`: Extracts the final answer or output generated by the agent and displays it in the Streamlit chat interface.

**In Summary**

This code sets up a Streamlit-based chatbot application that utilizes a ReAct agent powered by a large language model. The agent can access external tools like arXiv and Wikipedia to gather information and provide comprehensive responses. The use of a prompt template enforces a clear and structured response format, while the Streamlit callback handler provides insights into the agent's reasoning process. The application is designed to be interactive, allowing users to engage in a conversation with the agent and receive informative answers to their questions.